In [1]:
import torch

In [2]:
class MyCell(torch.nn.Module):
  def __init__(self):
    super(MyCell, self).__init__()
    self.linear = torch.nn.Linear(4,4)

  def forward(self, x, h):
    new_h = torch.tanh(self.linear(x) + h)
    return new_h, new_h

my_cell = MyCell()
print(my_cell)
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x,h))

MyCell(
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[0.8067, 0.2245, 0.1021, 0.6657],
        [0.5126, 0.4987, 0.3034, 0.5233],
        [0.3233, 0.6483, 0.0084, 0.6364]], grad_fn=<TanhBackward>), tensor([[0.8067, 0.2245, 0.1021, 0.6657],
        [0.5126, 0.4987, 0.3034, 0.5233],
        [0.3233, 0.6483, 0.0084, 0.6364]], grad_fn=<TanhBackward>))


In [5]:
my_cell = MyCell()
# print(my_cell)
x = torch.rand(3, 4)
h = torch.rand(3, 4)
traced_cell = torch.jit.trace(my_cell, (x,h))
print(traced_cell)
traced_cell(x, h)
# print(my_cell(x,h))

MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
)


(tensor([[ 0.5511,  0.7736, -0.1522,  0.3575],
         [ 0.6297,  0.6705,  0.1698, -0.0884],
         [ 0.7686,  0.7256,  0.2532, -0.2434]],
        grad_fn=<DifferentiableGraphBackward>),
 tensor([[ 0.5511,  0.7736, -0.1522,  0.3575],
         [ 0.6297,  0.6705,  0.1698, -0.0884],
         [ 0.7686,  0.7256,  0.2532, -0.2434]],
        grad_fn=<DifferentiableGraphBackward>))

In [6]:
print(traced_cell.graph)

graph(%self.1 : __torch__.___torch_mangle_7.MyCell,
      %input : Float(3:4, 4:1),
      %h : Float(3:4, 4:1)):
  %19 : __torch__.torch.nn.modules.linear.___torch_mangle_6.Linear = prim::GetAttr[name="linear"](%self.1)
  %21 : Tensor = prim::CallMethod[name="forward"](%19, %input)
  %12 : int = prim::Constant[value=1]() # <ipython-input-2-72193c1a2952>:7:0
  %13 : Float(3:4, 4:1) = aten::add(%21, %h, %12) # <ipython-input-2-72193c1a2952>:7:0
  %14 : Float(3:4, 4:1) = aten::tanh(%13) # <ipython-input-2-72193c1a2952>:7:0
  %15 : (Float(3:4, 4:1), Float(3:4, 4:1)) = prim::TupleConstruct(%14, %14)
  return (%15)



In [7]:
print(traced_cell.code)


def forward(self,
    input: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = torch.add((self.linear).forward(input, ), h, alpha=1)
  _1 = torch.tanh(_0)
  return (_1, _1)



* TorchScript 代码可以在其自己的解释器中调用，该解释器基本上是受限制的 Python 解释器。 该解释器不获取全局解释器锁定，因此可以在同一实例上同时处理许多请求。
* 这种格式允许我们将整个模型保存到磁盘上，然后将其加载到另一个环境中，例如在以 
Python 以外的语言编写的服务器中
* TorchScript 为我们提供了一种表示形式，其中我们可以对代码进行编译器优化以提供更有效的执行
* TorchScript 允许我们与许多后端/设备运行时进行交互，与单个操作员相比，它们要求更广泛的程序视图。

In [10]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell.code)

def forward(self,
    input: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = self.dg
  _1 = (self.linear).forward(input, )
  _2 = (_0).forward(_1, )
  _3 = torch.tanh(torch.add(_1, h, alpha=1))
  return (_3, _3)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
scripted_gate = torch.jit.script(MyDecisionGate())

my_cell = MyCell(scripted_gate)
traced_cell = torch.jit.script(my_cell)
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = (self.dg).forward((self.linear).forward(x, ), )
  new_h = torch.tanh(torch.add(_0, h, alpha=1))
  return (new_h, new_h)



In [9]:
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell(x, h)

(tensor([[ 0.7327,  0.4937,  0.0309, -0.2531],
         [ 0.8202,  0.2698,  0.6508,  0.1111],
         [ 0.7287,  0.6272,  0.3668,  0.1286]],
        grad_fn=<DifferentiableGraphBackward>),
 tensor([[ 0.7327,  0.4937,  0.0309, -0.2531],
         [ 0.8202,  0.2698,  0.6508,  0.1111],
         [ 0.7287,  0.6272,  0.3668,  0.1286]],
        grad_fn=<DifferentiableGraphBackward>))

In [12]:
class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.trace(MyCell(scripted_gate), (x, h))

    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.cell(xs[i], h)
        return y, h

rnn_loop = torch.jit.script(MyRNNLoop())
print(rnn_loop.code)

def forward(self,
    xs: Tensor) -> Tuple[Tensor, Tensor]:
  h = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  y = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  y0 = y
  h0 = h
  for i in range(torch.size(xs, 0)):
    _0 = (self.cell).forward(torch.select(xs, 0, i), h0, )
    y1, h1, = _0
    y0, h0 = y1, h1
  return (y0, h0)



In [13]:
class WrapRNN(torch.nn.Module):
    def __init__(self):
        super(WrapRNN, self).__init__()
        self.loop = torch.jit.script(MyRNNLoop())

    def forward(self, xs):
        y, h = self.loop(xs)
        return torch.relu(y)

traced = torch.jit.trace(WrapRNN(), (torch.rand(10, 3, 4)))
print(traced.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  _0, h, = (self.loop).forward(argument_1, )
  return torch.relu(h)



保存和加載模型

In [14]:
traced.save('wrapped_rnn.zip')

loaded = torch.jit.load('wrapped_rnn.zip')

print(loaded)
print(loaded.code)

RecursiveScriptModule(
  original_name=WrapRNN
  (loop): RecursiveScriptModule(
    original_name=MyRNNLoop
    (cell): RecursiveScriptModule(
      original_name=MyCell
      (dg): RecursiveScriptModule(original_name=MyDecisionGate)
      (linear): RecursiveScriptModule(original_name=Linear)
    )
  )
)
def forward(self,
    argument_1: Tensor) -> Tensor:
  _0, h, = (self.loop).forward(argument_1, )
  return torch.relu(h)

